In [11]:
import spacy
import re
import numpy as np
import math
import random
import time

# import pickle # use for saving and loading variables calculated by function step2_indexing,
#               # so that in the next time there will be no need to run step2_indexing again


def queryToTokens(queryString,sp=spacy.load('en_core_web_sm')):
    tokens = []
    sentence = sp(queryString)
    for token in sentence:
        if not token.is_stop and not token.is_punct and not token.is_space and not token.like_url:
            if (not token.is_alpha or not token.is_ascii) and not token.is_digit:
                result = re.findall(r'[a-z]+', token.lemma_)
                for i in result:
                    if len(i) > 1:
                        tokens.append(i)
                continue
            if len(token.text) > 1:
                tokens.append(token.lemma_)
    return tokens

def step1_preprocessing():
    file = open("./Trec_microblog11.txt", mode='r', encoding='UTF-8-sig')
    token_array = []
    sp = spacy.load('en_core_web_sm')
    line = file.readline().lower()
    count = 0 
    while line:
        count+=1
        if (count%1000 == 0):
            print("Processing line {}".format(str(count)),end="\r")
        tokens = queryToTokens(line,sp)
        token_array.append(tokens)
        line = file.readline().lower()
    file.close()
    print("\nFinished preprocessing!")
    return(token_array)



def step2_indexing(token_array):
    corpus_size = len(token_array)
    # build vocabulary list
#     start = time.time()
#     vocabulary = []
#     for doc in token_array:
#         for token in doc[1:]:
#             if not token in vocabulary:
#                 vocabulary.append(token)      
#     print("method1:%ss"%(time.time()-start)) #91.39s
    
    
#     start = time.time()
    vocabulary=set()
    for doc in token_array:
        for token in doc[1:]:
            vocabulary.add(token)
    vocabulary=list(vocabulary)
#     print("method2:%ss"%(time.time()-start)) #0.1s

    # initialize inverted index
    inverted_index = []
    for term in vocabulary:
        entry = []
        entry.append([term, 0])
        entry.append([])
        inverted_index.append(entry)

    # build inverted index
    doc_count = 0
    max_tf = 0
    for doc in token_array:
        docno = doc[0] # document number
        token_list = doc[1:]
        # calculate the number of occurences of each token in a document by using dictionary
        dictionary = {}
        for key in token_list:
            dictionary[key] = dictionary.get(key, 0) + 1

        # update inverted index by the information of the document
        for key in dictionary:
            tf = dictionary[key]
            doc_num_tf_pair = [docno, tf]
            key_index = vocabulary.index(key)
            inverted_index[key_index][1].append(doc_num_tf_pair)
            # update df
            inverted_index[key_index][0][1] += 1
            # update max tf
            if tf > max_tf:
                max_tf = tf

        doc_count += 1
        if doc_count % 1000 == 0:
            print("Finished processing {} documents".format(doc_count),end="\r")
            
    print("Finished processing {} documents".format(doc_count),end="\r")
    print()
    vocabulary_size = len(vocabulary)
    print("Finished building inverted index,")
    print("The size of vocabulary is:", vocabulary_size)

    return inverted_index, vocabulary, max_tf



def step3_retrieval_and_ranking(token_array, inverted_index, vocabulary, max_tf, doc_docno_search_list, query_token_list):
    # find the limited set of documents that contain at least one of the query words
    related_document_number_list = []
    for query_token in query_token_list:
        query_token_index = vocabulary.index(query_token)
        doc_num_tf_pair_list = inverted_index[query_token_index][1]
        for doc_num_tf_pair in doc_num_tf_pair_list:
            doc_num = doc_num_tf_pair[0]
            if not doc_num in related_document_number_list:
                related_document_number_list.append(doc_num)

    # build limited corpus
    related_document_location_list = []
    for docno in related_document_number_list:
        related_document_location_list.append(doc_docno_search_list.index(docno))

    limited_corpus = []
    for location in related_document_location_list:
        limited_corpus.append(token_array[location])

    # build new vocabulary list
    new_vocabulary = []
    for doc in limited_corpus:
        for token in doc[1:]:
            if not token in new_vocabulary:
                new_vocabulary.append(token)

    # calculate idf list
    corpus_size = len(token_array)
    new_vocabulary_idf_list = []
    for term in new_vocabulary:
        term_index = vocabulary.index(term)
        df = inverted_index[term_index][0][1]
        idf = math.log(corpus_size / df, 2)
        new_vocabulary_idf_list.append(idf)

    # build document-docno search list for the limited corpus
    limited_corpus_doc_docno_search_list = []
    for doc in limited_corpus:
        limited_corpus_doc_docno_search_list.append(doc[0])

    # build tf-idf matrix
    limited_corpus_size = len(limited_corpus)
    new_vocabulary_size = len(new_vocabulary)
    tf_idf_matrix = np.zeros((limited_corpus_size, new_vocabulary_size), dtype=np.float)

    for query_token in query_token_list:
        query_token_index = vocabulary.index(query_token)
        doc_num_tf_pair_list = inverted_index[query_token_index][1]
        column_num = new_vocabulary.index(query_token)
        idf = new_vocabulary_idf_list[column_num]
        for doc_num_tf_pair in doc_num_tf_pair_list:
            docno = doc_num_tf_pair[0]
            tf = doc_num_tf_pair[1]
            row_num = limited_corpus_doc_docno_search_list.index(docno)
            # normalize term frequency (tf) across the entire corpus
            # and here we put the process of building term frequency matrix and
            # tf-idf matrix together, to save the time for processing each query
            tf_idf_matrix[row_num][column_num] = (tf / max_tf) * idf

    # # multiply the tf scores by the idf values of each term,
    # # obtaining the following tf-idf matrix
    # tf_idf_matrix = np.zeros((limited_corpus_size, new_vocabulary_size), dtype=np.int)
    # for column_num in range(new_vocabulary_size):
    #     idf = new_vocabulary_idf_list[column_num]
    #     for row_num in range(limited_corpus_size):
    #         tf = tf_matrix[row_num][column_num]
    #         tf_idf_matrix[row_num][column_num] = tf * idf


    # calculate the tf-idf vector for the query
    query_tf_idf_vector = np.zeros((1, new_vocabulary_size), dtype=np.float)
    # first calculate the number of occurences of each token in the query by using dictionary
    dict = {}
    for key in query_token_list:
        dict[key] = dict.get(key, 0) + 1
    # find the maximum token frequency of the query
    query_max_tf = 0
    for key in dict:
        tf = dict[key]
        if tf > query_max_tf:
            query_max_tf = tf
    # calculate the query's tf-idf vector
    for key in dict:
        tf = dict[key]
        key_index = new_vocabulary.index(key)
        idf = new_vocabulary_idf_list[key_index]
        # a modified tf-idf weighting scheme w_iq = (0.5 + 0.5 tf_iq)∙idf_i
        query_tf_idf_vector[0][key_index] = (0.5 + 0.5 * (tf / query_max_tf)) * idf
        # # traditional tf-idf weighting scheme
        # query_tf_idf_vector[0][key_index] = (tf / query_max_tf) * idf

    # compute the similarity scores between a query and each document using cosine
    # save the result into a dictionary
    similarity_dictionary = {}
    query_tf_idf_vector_length = np.linalg.norm(query_tf_idf_vector)
    query_tf_idf_vector = query_tf_idf_vector.flatten()
    for row_num in range(limited_corpus_size):
        docno = limited_corpus_doc_docno_search_list[row_num]
        document_tf_idf_vector = tf_idf_matrix[row_num]
        cosSim = sum(document_tf_idf_vector * query_tf_idf_vector) / (np.linalg.norm(document_tf_idf_vector) * query_tf_idf_vector_length)
        similarity_dictionary[docno] = cosSim

    return similarity_dictionary

def loadQueries(path:"String of test query path")->"Dict in the format {queryNum('B001'):queryString('BBC World Service staff cuts')}":
    title = []
    num = []
    with open(path, "r") as f:  
        for line in f:        
            if line[:7] == "<title>":
                title.append(line[8:-10])
            elif line[:5] == "<num>": 
                num.append(line[15:-8])
    return dict(zip(num,title))

def loadResult(token_array, inverted_index,vocabulary, max_tf, doc_docno_search_list, queries)->"Dict{queryNum:Dict_DesendingScoreOrder{tweetid:score}}":
    returnValue = {}
    timeRecording=[]
    for num, query in queries.items():
        print("processing query {} of {}".format((int)(num[2:].lstrip('0')),len(queries)),end="\r")
        #####preprocess query
        sp = spacy.load('en_core_web_sm')
        query_token_list = queryToTokens(query, sp)
        start=time.time()
        result = step3_retrieval_and_ranking(token_array, inverted_index, vocabulary, max_tf, doc_docno_search_list, query_token_list)
        timeRecording.append(time.time()-start)
        sortedResult=dict(sorted(result.items(), key=lambda result:result[1], reverse=True))       
        returnValue[(int)(num[2:].lstrip('0'))] = dict(zip(list(sortedResult.keys())[:1000],sortedResult.values()))
    print()
    avgTime=round(sum(timeRecording)/len(timeRecording),2)
    print("Average time for each query processing: %ss"%(avgTime))
    return returnValue
    
    

def loadExpectedResult(expectedPath:"path of Trec_microblog11-qrels.txt")->"Dict{QueryNum:Set{all relevant ids}}":
    title = []
    num = []
    lastQueryNum = 0
    returnValue = {}
    with open(expectedPath, "r") as ef:  
        for line in ef: 
            lst = line[:-1].split("\t") #[queryNum, 0, id, relevant]
            currentQueryNum = int(lst[0])
            if (currentQueryNum != lastQueryNum):
                returnValue[currentQueryNum] = set() # add an empty set in the dict
            lastQueryNum = currentQueryNum
            if lst[3] == '1': # if relevant          
                returnValue[currentQueryNum].add(lst[2]) # add id to the set
                
    return returnValue


def saveResults(result,queries,outputPath,debug=False):
    '''
    step 4
    '''
    if (debug==True):
        outputPath = 'output/Result_{}.txt'.format(str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))
    with open(outputPath, 'w+') as f:
        f.write('topic_id Q0 docno rank score tag\n')
        for num, idScore in result.items():
            rank = 1
            for tweetId,score in idScore.items():
                f.write("{} {} {} {} {} {}".format(num, 'Q0', tweetId, rank, score,'myRun\n'))
                rank+=1
    return

def computeMAP(result:"Dict{queryNum:tweetid list in decending order}",expect:"Dict{QueryNum:Set{all relevant ids}}",
               first10:"set True if compute AP only for first 10"=False) -> "MAP or P@10":
    '''
    step 5
    '''
    APs=[]
    for queryNum,resultScore in result.items():
        
        expectForTheQuery = expect[queryNum]
        ids = list(resultScore.keys())
        relevant = 1
        relevantDocument = []
        
        for i in range(1, min(11, len(ids)+1) if first10 else len(ids)+1):
            if ids[i-1] in expectForTheQuery:
                relevantDocument.append(relevant/i)
                relevant+=1
        if len(relevantDocument) != 0:
            AP = sum(relevantDocument) / len(relevantDocument)
        else:
            AP = 0
        APs.append(round(AP, 4))
        
    MAP = sum(APs) / len(APs)
    return round(MAP, 4)

def tweetIdToString(tweetIdString, tweetPath):
    '''
    input:34952194402811904,path   output:Save BBC World Service from Savage Cuts http://www.petitionbuzz.com/petitions/savews
    '''
    tweetIdString = str(tweetIdString)
    with open(tweetPath, "r") as f:  
        for line in f: 
            line = line.strip('\ufeff')
            if((line.split("\t"))[0]==tweetIdString):
                return (line.split("\t"))[1][:-1] #[:-1] to remove '\n'
    return

def getResultTweet(queryNumInt,result, queries, tweetPath, n=10):
    '''
    2,result,queries,tweetPath -> first 10 tweet
    '''
    queryNum = 'B'+(3-len(str(queryNumInt)))*'0'+str(queryNumInt)
    print("Query %s:"%("number "+str(queryNumInt)),queries[queryNum],end="\n\n")
    tweetList = []
    tweetIds = list(result[queryNumInt].keys())[:n]
    for i in range(1,n+1):
        tweetList.append(tweetIdToString(str(tweetIds[i-1]), tweetPath))
        print("Rank %s: %s"%(str(i), tweetIdToString(str(tweetIds[i-1]), tweetPath)))
    
    return tweetList


In [12]:
token_array = step1_preprocessing()

Processing line 45000
Finished preprocessing!


In [13]:
inverted_index, vocabulary, max_tf = step2_indexing(token_array)
doc_docno_search_list = [i[0] for i in token_array] # build document-docno search list

Finished processing 45899 documents
Finished building inverted index,
The size of vocabulary is: 56428


In [14]:
tweetPath = "./Trec_microblog11.txt"
expectedPath = "./Trec_microblog11-qrels.txt"
queryPath = "./topics_MB1-49.txt"
outputPath = "./output/Result.txt"
queries = loadQueries(queryPath)

In [15]:
result = loadResult(token_array, inverted_index, vocabulary, max_tf, doc_docno_search_list, queries)

processing query 49 of 49
Average time for each query processing: 7.46s


In [16]:
expect = loadExpectedResult(expectedPath)

In [17]:
saveResults(result,queries,outputPath)

In [18]:
print("MAP: %s"%computeMAP(result,expect))
print("P@10: %s"%computeMAP(result,expect,first10=True))

MAP: 0.2195
P@10: 0.3326


In [19]:
tweetResult1 = getResultTweet(3, result, queries, tweetPath)
print('\n\n%s\n\n'%('*'*100))
tweetResult2 = getResultTweet(10, result, queries, tweetPath)

Query number 3: Haiti Aristide return

Rank 1: Former Haitian President Aristide has been issued with a new passport enabling him to end exile and return to Haiti, from AFP
Rank 2: Former Haitian President Aristide has been issued with a new passport enabling him to end exile and return to Haiti, from AFP
Rank 3: Good Lord RT @BBCBreaking: Former Haitian President Aristide issued with a new passport enabling him to end exile, return to Haiti, from AFP
Rank 4: Haiti's former president Jean-Bertrand Aristide vows to return http://gu.com/p/2nvx3/tf
Rank 5: If #Aristide returned to #Haiti, would it change anything? Would it create democracy?
Rank 6: Will NGOs & missionaries recognize #Haiti and #Haitians after Aristide returns? When the "helpless" are empowered then what?
Rank 7: Haiti allows ex-president Aristide's return http://english.aljazeera.net/news/americas/2011/02/2011217025580425.html … from @ajenglish (Can Haitian politics get any more interesting?)
Rank 8: #MIAMI Haiti to issue

In [20]:
# doc_docno_search_list = []
# for doc in token_array:
#     doc_docno_search_list.append(doc)

# # example query token list
# example_query_token_list = ['bbc', 'world', 'service', 'staff', 'cuts']


# sim_dict = step3_retrieval_and_ranking(token_array, inverted_index,
#                                        vocabulary, max_tf, doc_docno_search_list, example_query_token_list)

In [21]:
import pickle

In [36]:
# saves
pickle.dump( token_array, open( "./saves/token_array.p", "wb" ) )
pickle.dump( inverted_index, open( "./saves/inverted_index.p", "wb" ) )
pickle.dump( vocabulary, open( "./saves/vocabulary.p", "wb" ) )
pickle.dump( max_tf, open( "./saves/max_tf.p", "wb" ) )
pickle.dump( doc_docno_search_list, open( "./saves/doc_docno_search_list.p", "wb" ) )
pickle.dump( result, open( "./saves/result.p", "wb" ) )


In [37]:
# load
token_array = pickle.load( open( "./saves/token_array.p", "rb" ) )
inverted_index = pickle.load( open( "./saves/inverted_index.p", "rb" ) )
vocabulary = pickle.load( open( "./saves/vocabulary.p", "rb" ) )
max_tf = pickle.load( open( "./saves/max_tf.p", "rb" ) )
doc_docno_search_list = pickle.load( open( "./saves/doc_docno_search_list.p", "rb" ) )
result = pickle.load( open( "./saves/result.p", "rb" ) )

In [38]:
# run
expect = loadExpectedResult(expectedPath)
saveResults(result,queries,outputPath)

In [39]:
print("MAP: %s"%computeMAP(result,expect))
print("P@10: %s"%computeMAP(result,expect,first10=True))

MAP: 0.2195
P@10: 0.3326


In [40]:
tweetResult1 = getResultTweet(3, result, queries, tweetPath)
print('\n\n%s\n\n'%('*'*100))
tweetResult2 = getResultTweet(10, result, queries, tweetPath)

Query number 3: Haiti Aristide return

Rank 1: Former Haitian President Aristide has been issued with a new passport enabling him to end exile and return to Haiti, from AFP
Rank 2: Former Haitian President Aristide has been issued with a new passport enabling him to end exile and return to Haiti, from AFP
Rank 3: Good Lord RT @BBCBreaking: Former Haitian President Aristide issued with a new passport enabling him to end exile, return to Haiti, from AFP
Rank 4: Haiti's former president Jean-Bertrand Aristide vows to return http://gu.com/p/2nvx3/tf
Rank 5: If #Aristide returned to #Haiti, would it change anything? Would it create democracy?
Rank 6: Will NGOs & missionaries recognize #Haiti and #Haitians after Aristide returns? When the "helpless" are empowered then what?
Rank 7: Haiti allows ex-president Aristide's return http://english.aljazeera.net/news/americas/2011/02/2011217025580425.html … from @ajenglish (Can Haitian politics get any more interesting?)
Rank 8: #MIAMI Haiti to issue

In [41]:
import os

In [42]:
print (os.getcwd())

/Users/yi/Desktop/IR-N
